## Set up google colab environment

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

In [1]:
import os
os.chdir('/content/drive/My Drive/Hyperspectral_Image_Classification')

In [2]:
from SGCNN_7_Utils import *
import scipy.io as sio

## Load Pavia Dataset - Source

In [3]:
uPavia = sio.loadmat('/content/drive/My Drive/Hyperspectral_Image_Classification/Datasets/PaviaU.mat')
gt_Pavia = sio.loadmat('/content/drive/My Drive/Hyperspectral_Image_Classification/Datasets/PaviaU_gt.mat')

In [4]:
data_source = uPavia['paviaU']
ground_truth_source = gt_Pavia['paviaU_gt']

In [5]:
data_source.shape

(610, 340, 103)

In [6]:
ground_truth_source.shape

(610, 340)

# Load Indian Pines dataset - Target

In [7]:
uIndianPines = sio.loadmat('/content/drive/My Drive/Hyperspectral_Image_Classification/Datasets/Indian_pines_corrected.mat')
gt_IndianPines = sio.loadmat('/content/drive/My Drive/Hyperspectral_Image_Classification/Datasets/Indian_pines_gt.mat')

In [8]:
data_target = uIndianPines['indian_pines_corrected']
ground_truth_target = gt_IndianPines['indian_pines_gt']

In [9]:
data_target.shape

(145, 145, 200)

In [10]:
ground_truth_target.shape

(145, 145)

## Distrubution of samples for each class in Source

In [11]:
class_distribution_source = pd.DataFrame(np.unique(ground_truth_source, return_counts = True))
class_distribution_source = class_distribution_source.transpose()
class_distribution_source.columns = ['class','samples']
class_distribution_source

,class,samples
0,0,164624
1,1,6631
2,2,18649
3,3,2099
4,4,3064
5,5,1345
6,6,5029
7,7,1330
8,8,3682
9,9,947


## Drop background class

In [12]:
classes_source , counts_source = np.unique(ground_truth_source, return_counts = True)
classes_source = classes_source[1:] ## Dropping classes with background
classes_source

array([1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

# Class distribution of samples in Target

In [13]:
class_distribution_target = pd.DataFrame(np.unique(ground_truth_target, return_counts = True))
class_distribution_target = class_distribution_target.transpose()
class_distribution_target.columns = ['class','samples']
class_distribution_target

,class,samples
0,0,10776
1,1,46
2,2,1428
3,3,830
4,4,237
5,5,483
6,6,730
7,7,28
8,8,478
9,9,20


## Dropping classes with small number of samples

In [14]:
classes_target , counts_target = np.unique(ground_truth_target, return_counts = True)
classes_target = classes_target[[2,3,5,6,8,10,11,12,14]] ## Dropping classes with small number of samples
classes_target

array([ 2,  3,  5,  6,  8, 10, 11, 12, 14], dtype=uint8)

## Source : Pavia

## Train model for samples extracted with different overlap ratios and a percent of  samples picked from each class to be present in the training set. 

## Model except the final fully connected layer is saved for transfer learning.

In [15]:
pretrain_results = pretrain_source_models(percentages = [60,75,90],
                                          classes = classes_source,
                                          cube_size = 20,
                                          overlap_ratios = [1],
                                          data = data_source,
                                          ground_truth = ground_truth_source,
                                          batch_size = 20,
                                          channels = 64,
                                          epochs = 50,
                                          Verbosity = 1,
                                          accuracies = [],
                                          learning_rate = 0.0001,
                                          source_dataset = 'pavia')


Model training starts for data with overlap ratio 1.0 and 60 percent samples from each class in training set 

Samples per class: [5975, 15062, 1742, 2854, 1345, 5029, 1330, 3682, 940]
Total number of samples is 37959.

unique classes in training set: [1 2 3 4 5 6 7 8 9]
Total number of samples in training set is 22774.
Samples per class in training set: [3585 9037 1045 1712  807 3017  798 2209  564]

unique classes in test set: [1 2 3 4 5 6 7 8 9]
Total number of samples in test set is 15185.
Samples per class in test set: [2390 6025  697 1142  538 2012  532 1473  376]

X_train => (22774, 20, 20, 64)
X_test  => (15185, 20, 20, 64)
y_train => (22774, 9)
y_test  => (15185, 9)

Group convolution output:  (None, 20, 20, 64)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 20, 20, 64)] 0 

In [16]:
pretrain_results

,Overlap_ratio,Training Samples,Test Samples,Training_Test_Split,Test_Accuracies
0,1,22774,15185,60,70.16
1,1,28465,9494,75,84.08
2,1,34159,3800,90,93.42


# Fine tune on Indian Pines

In [17]:
transfer_results, confusion_matrixes = transfer_learning(percentages = [60,75,90],
                                                        source_dataset = 'pavia',
                                                        target_dataset = 'indian_pines',
                                                        data = data_target,
                                                        ground_truth = ground_truth_target,
                                                        classes = classes_target,
                                                        overlap_ratios = [1],
                                                        channels = 64,
                                                        cube_size = 20,
                                                        learning_rate = 0.0001,
                                                        epochs = 50,
                                                        batch_size = 20)


Model training starts for data with overlap ratio 1.0 and 60 percent samples from each class in training set 

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 20, 20, 64)] 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 20, 20, 64)   4160        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 20, 20, 64)   256         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 20, 20, 64)   0           batch_normaliz

# Transfer Learning results

In [18]:
transfer_results

,Overlap_ratio,Training Samples,Test Samples,Training_Test_Split,Test_Accuracies
0,1,4740,3166,60,60.99
1,1,5927,1979,75,60.79
2,1,7112,794,90,63.10


# Classification accuracies per class for each model

In [19]:
for cm in confusion_matrixes:
  print(cm)

                 1    2    3    4  ...    7    8    9  classfication_accuracies
0               55   15    0   42  ...  381    0    0                     10.04
1                0  141    5   11  ...   35    0   18                     67.14
2                0    0    0   88  ...    0    0   42                       0.0
3                0    0   44  226  ...    0    0   22                      77.4
4                0    0    0    0  ...    0    0    0                     100.0
5                2    0    0    0  ...  126    0    0                     61.79
6                0  110    4   63  ...  713    0    0                     80.11
7               80    0    0    9  ...   75   12    0                      6.52
8                0    0    0    0  ...    0    0  434                     100.0
Total Samples  548  210  130  292  ...  890  184  434                         -

[10 rows x 10 columns]
                 1    2   3    4  ...    7    8    9  classfication_accuracies
0               1